In [1]:
import pandas as pd
import numpy as np
import json
import os
import urllib3
import csv
import logging
import re
import uuid
import shortuuid
from pathlib import Path
from idmc_utils import generate_taskflow, package_import
from urllib.parse import quote

# Set the pandas options
pd.set_option('display.max_columns', None)

# Initialise the job

In [2]:
# Initialise the log file
logging.basicConfig(
    filename='logs/console.log',
    level=logging.DEBUG,
    format='%(asctime)s:%(levelname)s:%(message)s',
)

In [3]:
# Read the config file
logging.info('Reading the config file')
with open('config/config.json', 'r') as infile:
    config = json.load(infile)


# Read the Inputs

In [4]:
# Read the execution plans
logging.info('Reading the execution plans')
dfPlans = pd.read_csv('in/plans.csv', dtype='str', encoding='utf-8', na_filter=False)
dfPlans['plan_step_order'] = dfPlans['plan_step_order'].astype(int)

# Lookup the Converted Mapping Task IDs

In [5]:
# Login to IDMC
logging.info('Logging into IDMC')
http = urllib3.PoolManager()

data = '{ "username": "' + config['idmc']['user'] + '", "password": "' + config['idmc']['password'] + '" }'

url = 'https://' + config['idmc']['host'] + '/saas/public/core/v3/login'
r = http.request(
    'POST', 
    url,
    timeout=3000,
    body=data,
    headers={
            'Accept': 'application/json',
            'Content-Type': 'application/json'
        }
    )
    
# Convert the response into a datframe
result = json.loads(r.data.decode('utf-8'))
sessionID = result['userInfo']['sessionId']


In [6]:
# Get the secure agent ID
logging.info('Getting the secure agent ID')
url = 'https://' + config['idmc']['pod'] + '.' + config['idmc']['host'] + '/saas/api/v2/runtimeEnvironment/name/' + quote(config['idmc']['agentGroupName'])
r = http.request(
    'GET', 
    url,
    timeout=3000,
    headers={
            'Accept': 'application/json',
            'icSessionId': sessionID
        }
    )
    
# Convert the response into a datframe
result = json.loads(r.data.decode('utf-8'))
agentGroupID = result['id']
agentGroupGUID = result['federatedId']
agentGroupName = config['idmc']['agentGroupName']

In [7]:
# Initialise the tasks data frame
logging.info('Getting a list of the mapping tasks')
skip = 0
limit = 200
i = 0
dfTasks = pd.DataFrame()

# Page through mapping task queries
while True:

    # Get a list of the mapping tasks
    url = 'https://' + config['idmc']['pod'] + '.' + config['idmc']['host'] + '/saas/public/core/v3/objects?q=type==%27MTT%27&limit=' + str(limit) + '&skip=' + str(skip)
    r = http.request(
        'GET', 
        url,
        timeout=3000,
        headers={
                'Accept': 'application/json',
                'INFA-SESSION-ID': sessionID
            }
        )
        
    # Convert the response into a datframe
    result = json.loads(r.data.decode('utf-8'))
    dfTmp = pd.json_normalize(result)
    dfResp = dfTmp.copy()
    taskCount = dfResp.iloc[0]['count']
    dfResp = dfResp['objects'].explode()
    dfResp = pd.DataFrame(dfResp)
    dfResp = pd.json_normalize(dfResp['objects'])
    dfTasks = pd.concat([dfTasks, dfResp], ignore_index=True)

    # Break if all records have been returned
    i = i + limit
    if i > taskCount:
        break



In [8]:
# Join the IDMC info onto the plans
dfResp['step_name'] = dfResp['path'].apply(lambda x: os.path.basename(x))
dfResp = dfResp[['step_name','id','path']]
dfResp = dfResp.rename(columns={'id': 'infa_id', 'path': 'infa_path'})
dfPlans = dfPlans.merge(dfResp, how='left', on='step_name')
dfPlans['agent_id'] = agentGroupID
dfPlans['agent_guid'] = agentGroupGUID
dfPlans['agent_name'] = agentGroupName
dfPlans['script_dir'] = config['local']['scriptsDir']
dfPlans['script_args'] = '' # TODO placeholder for any args that need to be passed to the step script

In [9]:
#TODO Delete this after initial testing is finished
dfPlans = dfPlans.head(1)
dfPlans

,plan_wid,plan_name,plan_inactive_flag,plan_step_wid,plan_step_order,plan_step_type,step_guid,step_wid,step_cmd,step_name,infa_id,infa_path,agent_id,agent_guid,agent_name,script_dir,script_args
0,201C9CC7C59D167A79A2E247C6189A67,Echo Employee Snapshot Oracle R12.1.3,N,4F418CAB76FCD515C5A7F3C94D552CF,0,REGULAR,4F418CAB76FCD515C5A7F3C94D552CF,1911f67f35d9a487283f503fc7ab2ac,SDE_ORA_EmployeeDailySnapshotFact_2,SDE_ORA_EmployeeDailySnapshotFact_2,a0Ks8uNXYKLg38LRNzw6gv,Default/SDE_ORA_EmployeeDailySnapshotFact_2,010SU125000000000002,cPbb2XLzpoweqfBP9W6kOJ,AUW487V7S3-AAD,C:\Informatica\scripts,


In [10]:
# Generate the unique identifiers
#dfPlans['dac2idmc_step_id'] = dfPlans.apply(lambda x: str(uuid.uuid4()).replace('-',''), axis=1)
dfPlans['dac2idmc_step_id'] = dfPlans.apply(lambda x: "task" + shortuuid.uuid()[:8], axis=1)
#map_order_uuid = { step_order: str(uuid.uuid4()).replace('-','') for step_order in dfPlans['plan_step_order'].unique() }
map_order_uuid = { step_order: "group" + shortuuid.uuid()[:8] for step_order in dfPlans['plan_step_order'].unique() }
dfPlans['dac2idmc_group_id'] = dfPlans['plan_step_order'].map(map_order_uuid)
dfPlans['dac2idmc_next_id'] = dfPlans['dac2idmc_step_id'].shift(-1)


In [11]:
dfPlans.head()

,plan_wid,plan_name,plan_inactive_flag,plan_step_wid,plan_step_order,plan_step_type,step_guid,step_wid,step_cmd,step_name,infa_id,infa_path,agent_id,agent_guid,agent_name,script_dir,script_args,dac2idmc_step_id,dac2idmc_group_id,dac2idmc_next_id
0,201C9CC7C59D167A79A2E247C6189A67,Echo Employee Snapshot Oracle R12.1.3,N,4F418CAB76FCD515C5A7F3C94D552CF,0,REGULAR,4F418CAB76FCD515C5A7F3C94D552CF,1911f67f35d9a487283f503fc7ab2ac,SDE_ORA_EmployeeDailySnapshotFact_2,SDE_ORA_EmployeeDailySnapshotFact_2,a0Ks8uNXYKLg38LRNzw6gv,Default/SDE_ORA_EmployeeDailySnapshotFact_2,010SU125000000000002,cPbb2XLzpoweqfBP9W6kOJ,AUW487V7S3-AAD,C:\Informatica\scripts,,taskKayuJ849,groupTYR3agag,None


In [12]:
# Log an error if any plans did not find an existing matching task
dfMissing = dfPlans[(dfPlans['infa_id'] == '') | (dfPlans['infa_id'].isna())].copy()
if len(dfMissing.index) > 0:
    logging.error('Some plans are missing a converted mapping task. Please see "out/missing_tasks.csv" for more details')
    dfMissing.to_csv('out/missing_tasks.csv', index=False, quoting=csv.QUOTE_ALL)

# Generate the Taskflows

In [13]:
#TODO add loop through execution plans
planIds = dfPlans['plan_wid'].unique()

for planId in planIds:

    # Get the plan for the current id
    dfPlan = dfPlans[dfPlans['plan_wid'] == planId].copy()

    # Generate the taskflow ID
    #taskflowID = str(uuid.uuid4()).replace('-','')
    taskflowID = shortuuid.uuid()
    taskflowName = dfPlan.iloc[0]['plan_name']
    taskflowName = re.sub(r'\W+', '_', taskflowName)

    logging.info(f'Create workspace tree "tmp/{ taskflowName }/Explore/Default"')

    # Create the workspace directories
    treePath = Path(f'tmp/{ taskflowName }/Explore/Default')
    treePath.mkdir(parents=True, exist_ok=True)

    logging.info(f'Generating taskflow "{ taskflowName }"')
    generate_taskflow.generate_taskflow(taskflowID, taskflowName, dfPlan)

    logging.info(f'Packaging taskflow "{ taskflowName }"')
    package_import.package_import(taskflowID, taskflowName, dfPlan)

    logging.info(f'Done!')

In [14]:
dfPlans.sort_values('plan_step_order')

,plan_wid,plan_name,plan_inactive_flag,plan_step_wid,plan_step_order,plan_step_type,step_guid,step_wid,step_cmd,step_name,infa_id,infa_path,agent_id,agent_guid,agent_name,script_dir,script_args,dac2idmc_step_id,dac2idmc_group_id,dac2idmc_next_id
0,201C9CC7C59D167A79A2E247C6189A67,Echo Employee Snapshot Oracle R12.1.3,N,4F418CAB76FCD515C5A7F3C94D552CF,0,REGULAR,4F418CAB76FCD515C5A7F3C94D552CF,1911f67f35d9a487283f503fc7ab2ac,SDE_ORA_EmployeeDailySnapshotFact_2,SDE_ORA_EmployeeDailySnapshotFact_2,a0Ks8uNXYKLg38LRNzw6gv,Default/SDE_ORA_EmployeeDailySnapshotFact_2,010SU125000000000002,cPbb2XLzpoweqfBP9W6kOJ,AUW487V7S3-AAD,C:\Informatica\scripts,,taskKayuJ849,groupTYR3agag,None


In [15]:
dfPlans.iloc[0]['dac2idmc_next_id']